In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import os
import re
import nltk
from nltk.stem import PorterStemmer
from collections import Counter
stemmer = PorterStemmer()

c:\Users\abjar\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Do some basic cleaning to handle missing values

In [2]:
path = kagglehub.dataset_download("lakshmi25npathi/online-retail-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\abjar\.cache\kagglehub\datasets\lakshmi25npathi\online-retail-dataset\versions\1


In [3]:
dataset_path = "C:/Users/abjar/.cache/kagglehub/datasets/lakshmi25npathi/online-retail-dataset/versions/1"

files = os.listdir(dataset_path)
print(files)

['online_retail_II.xlsx']


In [4]:
file_path = "C:/Users/abjar/.cache/kagglehub/datasets/lakshmi25npathi/online-retail-dataset/versions/1/online_retail_II.xlsx"

df = pd.read_excel(file_path)

In [5]:
print(df.head())

  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom  


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [7]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,525461.000000,525461,525461.000000,417534.000000
mean,10.337667,2010-06-28 11:37:36.845017856,4.688834,15360.645478
min,-9600.000000,2009-12-01 07:45:00,-53594.360000,12346.000000
25%,1.000000,2010-03-21 12:20:00,1.250000,13983.000000
50%,3.000000,2010-07-06 09:51:00,2.100000,15311.000000
75%,10.000000,2010-10-15 12:45:00,4.210000,16799.000000
max,19152.000000,2010-12-09 20:01:00,25111.090000,18287.000000
std,107.424110,NaN,146.126914,1680.811316


In [8]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [9]:
# Create a copy of the original DataFrame
df_cleaned = df.copy()

# Drop rows where 'CustomerID' is missing
df_cleaned = df_cleaned.dropna(subset=['Customer ID'])

# Fill missing 'Description' values with "Unknown"
df_cleaned['Description'].fillna('Unknown', inplace=True)

# Verify changes
print(df_cleaned.isnull().sum())

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64


C:\Users\abjar\AppData\Local\Temp\ipykernel_20972\3597769721.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['Description'].fillna('Unknown', inplace=True)


In [10]:
print(df_cleaned.duplicated().sum())

6771


In [11]:
df_cleaned[df_cleaned.duplicated()].head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
371,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
383,489517,22130,PARTY CONE CHRISTMAS DECORATION,6,2009-12-01 11:34:00,0.85,16329.0,United Kingdom
384,489517,22319,HAIRCLIPS FORTIES FABRIC ASSORTED,12,2009-12-01 11:34:00,0.65,16329.0,United Kingdom
385,489517,21913,VINTAGE SEASIDE JIGSAW PUZZLES,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
386,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
390,489517,84951A,S/4 PISTACHIO LOVEBIRD COASTERS,1,2009-12-01 11:34:00,2.55,16329.0,United Kingdom
391,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01 11:34:00,1.95,16329.0,United Kingdom
394,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
657,489529,22028,PENNY FARTHING BIRTHDAY CARD,12,2009-12-01 11:51:00,0.42,17984.0,United Kingdom
658,489529,22036,DINOSAUR BIRTHDAY CARD,12,2009-12-01 11:51:00,0.42,17984.0,United Kingdom


In [12]:
# Check duplicates without 'InvoiceNo' (to see if it's the same product/customer)
df_cleaned.duplicated(subset=['Customer ID', 'StockCode', 'Quantity', 'InvoiceDate']).sum()


np.int64(6816)

In [13]:
df_cleaned = df_cleaned.drop_duplicates()
print(f"Remaining rows after dropping duplicates: {len(df_cleaned)}")

Remaining rows after dropping duplicates: 410763


In [14]:
print(df_cleaned.duplicated().sum())  # Should now be 0

0


now that data cleaning has been done lets create the following features:
  *   Revenue
  *   DayOfWeek: to analyze sales trends by weekdays.
  *   TotalRevenue for each CustomerID
  *   Most popular product based on Revenue.
  *   Ordersize by summing Quantity for each InvoiceNo

we will start with creating the revenue feature

In [15]:
df_cleaned.loc[:,"Revenue"] = df_cleaned["Quantity"] * df_cleaned["Price"]
df_cleaned.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


then let us do the the day of week feature 

In [16]:
# day of week
df_cleaned.loc[:,"invoiceDate"] = pd.to_datetime(df_cleaned["InvoiceDate"])
df_cleaned.loc[:,"DayOfWeek"] = df_cleaned["invoiceDate"].dt.day_name()
df_cleaned.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue,invoiceDate,DayOfWeek
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,2009-12-01 07:45:00,Tuesday
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009-12-01 07:45:00,Tuesday
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009-12-01 07:45:00,Tuesday
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,2009-12-01 07:45:00,Tuesday
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,2009-12-01 07:45:00,Tuesday


lets do the revenue per customer, this will not be added to the dataframe, but instead will help me categorize the customers at a later point 

In [17]:
# revenue per customer
customer_revenue = df_cleaned.groupby("Customer ID")["Revenue"].sum().reset_index()
customer_revenue.rename(columns={"Revenue":"TotalRevenue"}, inplace=True)
customer_revenue.head()


,Customer ID,TotalRevenue
0,12346.0,-51.74
1,12347.0,1323.32
2,12348.0,222.16
3,12349.0,2646.99
4,12351.0,300.93


we will create order size per order id, i have decided to not add this to the dataframe as it will create a lot of redundancy. later i will categorize multi-item invoices with a boolean.

In [18]:
order_size = df_cleaned.groupby("Invoice")["Quantity"].sum().reset_index()
order_size.rename(columns={"Quantity": "OrderSize"},inplace=True)
order_size.head()

,Invoice,OrderSize
0,489434,166
1,489435,60
2,489436,193
3,489437,145
4,489438,826


Alright, this is the current dataframe, with all the new features added to it:

In [19]:
df_cleaned.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue,invoiceDate,DayOfWeek
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,2009-12-01 07:45:00,Tuesday
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009-12-01 07:45:00,Tuesday
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009-12-01 07:45:00,Tuesday
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,2009-12-01 07:45:00,Tuesday
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,2009-12-01 07:45:00,Tuesday


now that the required features have been created let us continue with doing som lambda functions to create a more detailed look

first lets have a look at segmenting customers into tiers based on TotalRevenue (e.g., "High", "Medium", "Low").

In [20]:
high_threshold = customer_revenue["TotalRevenue"].quantile(0.75)  
low_threshold = customer_revenue["TotalRevenue"].quantile(0.25)   

df_cleaned.loc[:,"CustomerTier"] = customer_revenue["TotalRevenue"].apply(
    lambda x: "High" if x >= high_threshold else ("Medium" if x >= low_threshold else "Low")
)
df_cleaned.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue,invoiceDate,DayOfWeek,CustomerTier
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,2009-12-01 07:45:00,Tuesday,Low
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009-12-01 07:45:00,Tuesday,Medium
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009-12-01 07:45:00,Tuesday,Low
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,2009-12-01 07:45:00,Tuesday,High
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,2009-12-01 07:45:00,Tuesday,Medium


next lets extract key information from Description and add them as columns (e.g., presence of specific keywords like "Gift" or "Discount"). i will do this by tokenizing and stemming the words to check with a list of keywords 


In [21]:
# Define key stemmed words to track
key_words = ["gift", "luxuri", "set", "discount"]

# Ensure descriptions are string type
df_cleaned['Description'] = df_cleaned['Description'].astype(str).str.lower()

# Function to check if a stemmed word is in a description
def contains_keyword(description, keyword):
    words = re.findall(r'\b\w+\b', description)  # Tokenize using regex
    stemmed_words = [stemmer.stem(word) for word in words]  # Stem words
    return 1 if keyword in stemmed_words else 0

# Apply function to create binary columns
for word in key_words:
    df_cleaned[f"Contains_{word}"] = df_cleaned['Description'].apply(lambda x: contains_keyword(x, word))

# Show sample results
print(df_cleaned[['Description', 'Contains_gift', 'Contains_luxuri', 'Contains_set', 'Contains_discount']].head(10))



                           Description  Contains_gift  Contains_luxuri  \
0  15cm christmas glass ball 20 lights              0                0   
1                   pink cherry lights              0                0   
2                  white cherry lights              0                0   
3         record frame 7" single size               0                0   
4       strawberry ceramic trinket box              0                0   
5           pink doughnut trinket pot               0                0   
6                  save the planet mug              0                0   
7   fancy font home sweet home doormat              0                0   
8                            cat bowl               0                0   
9       dog bowl , chasing ball design              0                0   

   Contains_set  Contains_discount  
0             0                  0  
1             0                  0  
2             0                  0  
3             0                  0  


now lets categorize transactions as "Small", "Medium", or "Large" based on Revenue.

In [22]:
low_threshold = df_cleaned["Revenue"].quantile(0.25)
high_threshold = df_cleaned["Revenue"].quantile(0.75)

df_cleaned["TransactionSize"] = df_cleaned["Revenue"].apply(
    lambda x: "Large" if x > high_threshold else ("Medium" if x >= low_threshold else "Small")
)

print(df_cleaned["TransactionSize"].value_counts())
df_cleaned[["Revenue", "TransactionSize"]].head(10)

TransactionSize
Medium    208785
Small     102641
Large      99337
Name: count, dtype: int64


,Revenue,TransactionSize
0,83.4,Large
1,81.0,Large
2,81.0,Large
3,100.8,Large
4,30.0,Large
5,39.6,Large
6,30.0,Large
7,59.5,Large
8,30.6,Large
9,45.0,Large


now we will detect seasonal items, right now we only have christmas items, but this can be expanded upon if needed. 


for this im using stemming and tokenization to flag the items

In [23]:

# Define Christmas-related keywords (original and stemmed)
christmas_keywords = ["christmas", "holiday", "santa", "festive", "snow", "reindeer", "winter", "noel"]

# Stem all Christmas-related words
stemmed_christmas_keywords = {stemmer.stem(word) for word in christmas_keywords}

# Function to tokenize and stem descriptions
def is_christmas_item(description):
    words = re.findall(r'\b\w+\b', description.lower())  # Tokenize
    stemmed_words = {stemmer.stem(word) for word in words}  # Stem words
    return 1 if any(word in stemmed_words for word in stemmed_christmas_keywords) else 0

# Apply function to create a new column
df_cleaned["IsChristmasItem"] = df_cleaned["Description"].apply(is_christmas_item)

# Check sample results
print(df_cleaned[["Description", "IsChristmasItem"]].head())

print(df_cleaned["IsChristmasItem"].value_counts())




                           Description  IsChristmasItem
0  15cm christmas glass ball 20 lights                1
1                   pink cherry lights                0
2                  white cherry lights                0
3         record frame 7" single size                 0
4       strawberry ceramic trinket box                0
IsChristmasItem
0    393754
1     17009
Name: count, dtype: int64


now lets classify customers as "Loyal", "Occasional", or "One-time" based on the number of purchases.

In [24]:
import pandas as pd

# Compute total orders per customer & classify in one step
df_cleaned = df_cleaned.merge(
    df_cleaned.groupby("Customer ID")["Invoice"]
    .nunique()
    .reset_index()
    .rename(columns={"Invoice": "TotalOrders"})
    .assign(CustomerCategory=lambda x: x["TotalOrders"].apply(lambda orders: 
        "Loyal" if orders > 10 else "Occasional" if orders >= 3 else "One-time")),
    on="Customer ID",
    how="left"
)

# Check results
print(df_cleaned[["Customer ID", "CustomerCategory"]].head(10))



   Customer ID CustomerCategory
0      13085.0       Occasional
1      13085.0       Occasional
2      13085.0       Occasional
3      13085.0       Occasional
4      13085.0       Occasional
5      13085.0       Occasional
6      13085.0       Occasional
7      13085.0       Occasional
8      13085.0       Occasional
9      13085.0       Occasional


Now lets identify the multi-item invoices and categorize them if theyre either single or multi-item

In [ ]:
import pandas as pd


# Compute Multi-Item Order flag in one step
df_cleaned = df_cleaned.merge(
    df_cleaned.groupby("Invoice")["StockCode"].agg(lambda x: 1 if x.nunique() > 1 else 0)
    .reset_index()
    .rename(columns={"StockCode": "MultiItemOrder"}),
    on="Invoice",
    how="left"
)



In [ ]:
print(df_cleaned[["Invoice", "MultiItemOrder"]].head(10))

  Invoice  MultiItemOrder
0  489434               1
1  489434               1
2  489434               1
3  489434               1
4  489434               1
5  489434               1
6  489434               1
7  489434               1
8  489435               1
9  489435               1


now that all features and lambda functions have been created lets create an ETL pipeline  

In [ ]:
def extract(dataset_path):

    file_path = dataset_path
    df = pd.read_excel(file_path)

    return df

def clean_data(df):
    df_cleaned = df.copy() 
    
    # Drop rows where 'CustomerID' is missing
    df_cleaned = df_cleaned.dropna(subset=['Customer ID'])

    # Fill missing 'Description' values with "Unknown"
    df_cleaned['Description'].fillna('Unknown', inplace=True)

    # drop duplicates 
    df_cleaned = df_cleaned.drop_duplicates()

    return df_cleaned

def transform(df_cleaned):
    # revenue per customer 
    df_cleaned.loc[:,"Revenue"] = df_cleaned["Quantity"] * df_cleaned["Price"]

    # day of week 
    df_cleaned.loc[:,"invoiceDate"] = pd.to_datetime(df_cleaned["InvoiceDate"])
    df_cleaned.loc[:,"DayOfWeek"] = df_cleaned["invoiceDate"].dt.day_name()
    
    # revenue per customer
    customer_revenue = df_cleaned.groupby("Customer ID")["Revenue"].sum().reset_index()
    customer_revenue.rename(columns={"Revenue":"TotalRevenue"}, inplace=True)

    # order size
    order_size = df_cleaned.groupby("Invoice")["Quantity"].sum().reset_index()
    order_size.rename(columns={"Quantity": "OrderSize"},inplace=True)

    high_threshold = customer_revenue["TotalRevenue"].quantile(0.75)  
    low_threshold = customer_revenue["TotalRevenue"].quantile(0.25)   

    df_cleaned.loc[:,"CustomerTier"] = customer_revenue["TotalRevenue"].apply(
        lambda x: "High" if x >= high_threshold else ("Medium" if x >= low_threshold else "Low")
    )

    # Define key stemmed words to track
    key_words = ["gift", "luxuri", "set", "discount"]

    # Ensure descriptions are string type
    df_cleaned['Description'] = df_cleaned['Description'].astype(str).str.lower()

    # Function to check if a stemmed word is in a description
    def contains_keyword(description, keyword):
        words = re.findall(r'\b\w+\b', description)  # Tokenize using regex
        stemmed_words = [stemmer.stem(word) for word in words]  # Stem words
        return 1 if keyword in stemmed_words else 0

    # Apply function to create binary columns
    for word in key_words:
        df_cleaned[f"Contains_{word}"] = df_cleaned['Description'].apply(lambda x: contains_keyword(x, word))

    # transaction size 
    low_threshold = df_cleaned["Revenue"].quantile(0.25)
    high_threshold = df_cleaned["Revenue"].quantile(0.75)

    df_cleaned["TransactionSize"] = df_cleaned["Revenue"].apply(
        lambda x: "Large" if x > high_threshold else ("Medium" if x >= low_threshold else "Small")
    )

    # Christmas related keywords 
    christmas_keywords = ["christmas, holiday, santa, festive, snow, reindeer, winter"]
    
    stemmed_christmas_keywords = {stemmer.stem(word) for word in christmas_keywords}

    # Function to tokenize and stem descriptions
    def is_christmas_item(description):
        words = re.findall(r'\b\w+\b', description.lower())  # Tokenize
        stemmed_words = {stemmer.stem(word) for word in words}  # Stem words
        return 1 if any(word in stemmed_words for word in stemmed_christmas_keywords) else 0

    # Apply function to create a new column
    df_cleaned["IsChristmasItem"] = df_cleaned["Description"].apply(is_christmas_item)

    df_cleaned = df_cleaned.merge(
        df_cleaned.groupby("Customer ID")["Invoice"]
        .nunique()
        .reset_index()
        .rename(columns={"Invoice": "TotalOrders"})
        .assign(CustomerCategory=lambda x: x["TotalOrders"].apply(lambda orders: 
            "Loyal" if orders > 10 else "Occasional" if orders >= 3 else "One-time")),
        on="Customer ID",
        how="left"
    )

    # Compute Multi-Item Order flag in one step
    df_cleaned = df_cleaned.merge(
        df_cleaned.groupby("Invoice")["StockCode"].agg(lambda x: 1 if x.nunique() > 1 else 0)
        .reset_index()
        .rename(columns={"StockCode": "MultiItemOrder"}),
        on="Invoice",
        how="left"
    )

    return df_cleaned

def load(df_cleaned, path):
    df_cleaned.to_csv("cleaned_data.csv", index=False) 

    return df_cleaned

def ETL(dataset_path):
    df = extract(dataset_path)
    df_cleaned = clean_data(df)
    df_cleaned = transform(df_cleaned)
    df_cleaned = load(df_cleaned, dataset_path)

    return df_cleaned    

In [ ]:
dataset_path = "C:/Users/abjar/.cache/kagglehub/datasets/lakshmi25npathi/online-retail-dataset/versions/1/online_retail_II.xlsx"
df = ETL(dataset_path)
df.head()